In [ ]:
%matplotlib inline

import datetime

import numpy as np
import pandas as pd
from pandas.plotting import autocorrelation_plot
from sklearn.metrics import mean_absolute_error, mean_squared_error

import matplotlib.pyplot as plt
import plotly.graph_objs as go
import statsmodels.api as sm
from plotly.offline import plot as py
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
def parser(x):
    return datetime.datetime.strptime('190'+x, '%Y-%m')

In [ ]:
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/shampoo.csv

In [ ]:
df_data = pd.read_csv('shampoo.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)

In [ ]:
df_data.plot()
plt.show()

In [ ]:
df_data.head()

In [ ]:
autocorrelation_plot(df_data)
plt.show()

In [ ]:
sm.graphics.tsa.plot_acf(df_data.values.squeeze(), lags=35)
plt.show()

#### ARIMA Model Selection 

In [ ]:
# fit model
model = ARIMA(np.asarray(df_data), order=(1,1,0))
model_fit = model.fit(disp=0)
print(model_fit.summary())

# plot residual errors
residuals = pd.Series(model_fit.resid)
residuals.plot()
plt.show()
residuals.plot(kind='kde')
plt.show()
print(residuals.describe())

In [ ]:
# fit model
model = ARIMA(np.asarray(df_data), order=(5,1,0))
model_fit = model.fit(disp=0)
print(model_fit.summary())

# plot residual errors
residuals = pd.Series(model_fit.resid)
residuals.plot()
plt.show()
residuals.plot(kind='kde')
plt.show()
print(residuals.describe())

#### ARIMA Model Backtesting 

In [ ]:
X = np.asarray(df_data)
size = int(len(X) * 0.66)

train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()

for t in range(len(test)):
    model = ARIMA(history, order=(5,1,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))
mse_error = mean_squared_error(test, predictions)
mae_error = mean_absolute_error(test, predictions)

print('Test MSE: %.3f' % mse_error)
print('Test MAE: %.3f' % mae_error)

# plot
plt.plot(test)
plt.plot(predictions, color='red')
plt.show()

#### SARIMAX Model Backtesting 

In [ ]:
X = np.asarray(df_data)
size = int(len(X) * 0.66)

train, test = X[0:size], X[size:len(X)]
history = [x for x in train]

predictions = list()

for t in range(len(test)):
    model_trend = SARIMAX(history, trend='c', order=(5,1,0))
    model_fit = model_trend.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))
mse_error = mean_squared_error(test, predictions)
mae_error = mean_absolute_error(test, predictions)

print('Test MSE: %.3f' % mse_error)
print('Test MAE: %.3f' % mae_error)

# plot
plt.plot(test)
plt.plot(predictions, color='red')
plt.show()

In [ ]:
fcast_year = model_fit.get_forecast(12)
print('Forecast:')
print(fcast_year.predicted_mean)
print('Confidence intervals:')
print(fcast_year.conf_int())

In [ ]:
# Get the forecast and intervals as lists

x = list(range(12))
x_rev = x[::-1]
y1 = list(fcast_year.predicted_mean)
y1_upper = list(fcast_year.conf_int()[:,1])
y1_lower = list(fcast_year.conf_int()[:,0])
y1_lower = y1_lower[::-1]

In [ ]:
trace1 = go.Scatter(
    x=x+x_rev,
    y=y1_upper+y1_lower,
    fill='tozerox',
    fillcolor='rgba(0,176,246,0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    name='Shampoo sales',
    showlegend=False,
)

In [ ]:
trace2 = go.Scatter(
    x=x,
    y=y1,
    line=dict(color='rgb(0,176,246)'),
    mode='lines',
    name='Shampoo sales',
)

In [ ]:
data = [trace1, trace2]

layout = go.Layout(
    paper_bgcolor='rgb(255,255,255)',
    plot_bgcolor='rgb(229,229,229)',
    xaxis=dict(
        gridcolor='rgb(255,255,255)',
        range=[0,11],
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(127,127,127)',
        ticks='outside',
        zeroline=False
    ),
    yaxis=dict(
        gridcolor='rgb(255,255,255)',
        showgrid=True,
        showline=False,
        showticklabels=True,
        tickcolor='rgb(127,127,127)',
        ticks='outside',
        zeroline=False
    ),
)

fig = go.Figure(data=data, layout=layout)

fig.update_layout(
    title="Shampo sales forecast for 12 months",
    xaxis_title="Months",
    yaxis_title="Shampo sales",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)

fig.show()

In [ ]:
py(fig, filename = 'Sahmpoo_sales_SARIMA.html', auto_open=False)